## Data Pipeline Steps
1. Data Extraction
2. Quality Assesement/checks
3. Data Cleaning/Transformation
4. Data Validation
5. Feature Engineering
6. Loading
7. Analysis and Insights

## Activating Virtual Enviroment on windows
 .venv\Scripts\activate

In [1]:
#importing libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine    
from dotenv import load_dotenv
import psycopg2

### Step1: Data Extraction

In [7]:
# Define proper data types for key columns like 'customer_id', 'phone_number', etc.
dtypes_dict = {
    'customer_id': 'str',
    'phone_number': 'str',
    'tower_id': 'str',
    'call_id': 'str'
   
}

In [10]:
df_cdr = pd.read_csv(r'C:\Users\David Ibanga\Data Engineering practicals\Teleconnect_Analytics\data\teleconnect_cdr_data.csv', dtype=dtypes_dict)

In [11]:
df_cdr.head()

,call_id,customer_id,phone_number,tower_id,call_type,call_duration_seconds,data_usage_mb,signal_strength_dbm,call_timestamp,call_success,revenue_naira,network_type,roaming
0,86634543-444c-4f38-aa1d-70df574593fa,CU000151,08169541064,INVALID_TW9116,SMS,2532.0,4666.74,-85.0,2024-04-30 04:30:43,True,842.74,4G,True
1,fe36e9b9-f944-4d9a-83b2-a1796fc5074c,CU000307,07027999942,TW0040,data,NaN,3281.08,-74.0,2024-04-08 10:37:21,True,126.07,5G,False
2,534acb40-196b-469b-8b20-dbbe23a6e898,CU001104,07011766356,TW0128,Voice,5506.0,3967.93,NaN,2024-05-04 16:11:44,0,400.31,5G,0
3,04b2ab83-5325-4dd0-9a46-6d767439f5a2,CU000869,09059923201,NaN,Voice,4610.0,4734.73,-113.0,2024-05-24 14:01:58,False,956.19,3g,True
4,7775e1be-3638-4d37-8ae9-e91fd5a966e5,CU000518,08091140319,TW0150,SMS,271.0,2995.66,-106.0,2024-04-07 20:54:53,False,740.63,3G,1


### Understanding our data structure

In [12]:
df_cdr.columns

Index(['call_id', 'customer_id', 'phone_number', 'tower_id', 'call_type',
       'call_duration_seconds', 'data_usage_mb', 'signal_strength_dbm',
       'call_timestamp', 'call_success', 'revenue_naira', 'network_type',
       'roaming'],
      dtype='object')

In [13]:
df_cdr.shape

(5137, 13)

### Step2- Data Quality Assessment/Checks

#check for completnesss, uniquess and consistency of data
### Checking for missing values

In [18]:
#missing values in the dataset
missing_values = df_cdr.isnull().sum()
missing_values

call_id                    0
customer_id              425
phone_number             426
tower_id                 431
call_type                 25
call_duration_seconds    427
data_usage_mb            434
signal_strength_dbm      431
call_timestamp            25
call_success              25
revenue_naira             25
network_type              25
roaming                   25
dtype: int64

### Checking for duplicates record

In [19]:
duplicates = df_cdr.duplicated().sum()
duplicates

np.int64(112)

In [23]:
if duplicates > 0:
    duplicated_rows = df_cdr[df_cdr.duplicated(keep=False)]
    print(duplicated_rows.head())

                                  call_id customer_id phone_number tower_id  \
108  45191a1c-1767-41fc-a3d3-31883483b53d    CU000001  09058984075      NaN   
109  45191a1c-1767-41fc-a3d3-31883483b53d    CU000001  09058984075      NaN   
125  d3b8e673-f066-4f99-b66b-effe4feaf277    CU000819  09039727843      NaN   
126  d3b8e673-f066-4f99-b66b-effe4feaf277    CU000819  09039727843      NaN   
177  a082047a-8441-47de-9258-d1c9bfa52bd9    CU001447  08099575199      NaN   

    call_type  call_duration_seconds  data_usage_mb  signal_strength_dbm  \
108     Voice                 1502.0        2063.49                -41.0   
109     Voice                 1502.0        2063.49                -41.0   
125       SMS                   26.0         146.76                -45.0   
126       SMS                   26.0         146.76                -45.0   
177      data                 3494.0        4549.78                -51.0   

          call_timestamp call_success  revenue_naira network_type ro

### Checking Duplicated data Types


In [22]:
df_cdr.dtypes

call_id                   object
customer_id               object
phone_number              object
tower_id                  object
call_type                 object
call_duration_seconds    float64
data_usage_mb            float64
signal_strength_dbm      float64
call_timestamp            object
call_success              object
revenue_naira            float64
network_type              object
roaming                   object
dtype: object

### Checking for inconsistent Values

In [25]:
negative_durations = df_cdr[df_cdr['call_duration_seconds'] < 0]
len(negative_durations)

87

In [26]:
# check network type consistency
df_cdr['network_type'].value_counts()

network_type
5G    907
4G    858
4g    854
2G    846
3G    825
3g    822
Name: count, dtype: int64

In [27]:
df_cdr['call_type'].value_counts()

call_type
sms      894
voice    866
data     846
Data     845
SMS      832
Voice    829
Name: count, dtype: int64

In [28]:
df_cdr['call_success'].value_counts()

call_success
N        678
0        669
False    664
Y        643
1        633
Yes      632
No       605
True     588
Name: count, dtype: int64

In [29]:
df_cdr['roaming'].value_counts()

roaming
No       881
True     872
False    868
1        847
0        834
Yes      810
Name: count, dtype: int64

In [32]:
# df_cdr['phone_number'].value_counts()
phone_lengths = df_cdr['phone_number'].str.len()
phone_lengths.value_counts()

phone_number
11.0    4253
9.0      233
10.0     163
14.0      62
Name: count, dtype: int64

### Step 3 - Data Cleaning And Transformation

In [33]:
# create a copy of original data for cleaning
df_clean = df_cdr.copy()

### Handle missing values

In [34]:
df_clean.head()

,call_id,customer_id,phone_number,tower_id,call_type,call_duration_seconds,data_usage_mb,signal_strength_dbm,call_timestamp,call_success,revenue_naira,network_type,roaming
0,86634543-444c-4f38-aa1d-70df574593fa,CU000151,08169541064,INVALID_TW9116,SMS,2532.0,4666.74,-85.0,2024-04-30 04:30:43,True,842.74,4G,True
1,fe36e9b9-f944-4d9a-83b2-a1796fc5074c,CU000307,07027999942,TW0040,data,NaN,3281.08,-74.0,2024-04-08 10:37:21,True,126.07,5G,False
2,534acb40-196b-469b-8b20-dbbe23a6e898,CU001104,07011766356,TW0128,Voice,5506.0,3967.93,NaN,2024-05-04 16:11:44,0,400.31,5G,0
3,04b2ab83-5325-4dd0-9a46-6d767439f5a2,CU000869,09059923201,NaN,Voice,4610.0,4734.73,-113.0,2024-05-24 14:01:58,False,956.19,3g,True
4,7775e1be-3638-4d37-8ae9-e91fd5a966e5,CU000518,08091140319,TW0150,SMS,271.0,2995.66,-106.0,2024-04-07 20:54:53,False,740.63,3G,1


In [40]:
#handle missing values for critical columns
df_clean =df_clean.dropna(subset=['customer_id', 'phone_number', 'call_duration_seconds'])

In [41]:
fill_values = {
    'call_duration_seconds': 0,
    'data_usage_mb': 0,
    'signal_strength_dbm': df_clean['signal_strength_dbm'].median(),
    'tower_id': 'unknown'
}
df_clean = df_clean.fillna(value=fill_values)

In [42]:
df_clean.isnull().sum()

call_id                  0
customer_id              0
phone_number             0
tower_id                 0
call_type                0
call_duration_seconds    0
data_usage_mb            0
signal_strength_dbm      0
call_timestamp           0
call_success             0
revenue_naira            0
network_type             0
roaming                  0
dtype: int64

In [ ]:
initial_rows =len(df_clean)
initial_rows

4000

In [44]:
# drop duplicates
df_clean = df_clean.drop_duplicates()

In [45]:
initial_rows =len(df_clean)
initial_rows

3912

In [46]:
#Handle negative call durations #loc is short form for loaction #iloc is index location
df_clean.loc[df_clean['call_duration_seconds'] < 0, 'call_duration_seconds'] = 0

In [47]:
negative_durations = df_clean[df_clean['call_duration_seconds'] < 0]
len(negative_durations)

0

### Standardizing Text formats

In [ ]:
#.str.title makes the first letter of each word uppercase and the rest lowercase
df_clean['call_type'] = df_clean['call_type'].str.lower().str.title()
df_clean['call_type'].value_counts()

call_type
Sms      1329
Data     1303
Voice    1280
Name: count, dtype: int64

In [49]:
df_clean['network_type'] = df_clean['network_type'].str.upper().str.title()
df_clean['network_type'].value_counts()

network_type
4G    1309
3G    1266
5G     705
2G     632
Name: count, dtype: int64

In [50]:
df_clean['call_success'].value_counts()

call_success
N        525
0        513
False    507
1        500
Y        486
Yes      481
No       454
True     446
Name: count, dtype: int64

In [51]:
#standardizing boolean columns
def standardize_boolean(value):
    if pd.isna(value):
        return None
    value = str(value).lower()
    if value in ['yes', 'true', '1']:
        return True
    elif value in ['no', 'false', '0']:
        return False
    else:
        return None


    

In [52]:
df_clean['call_success']=df_clean['call_success'].apply(standardize_boolean)
df_clean['call_success'].value_counts()

call_success
False    1474
True     1427
Name: count, dtype: int64

In [53]:
df_clean['roaming']=df_clean['roaming'].apply(standardize_boolean)
df_clean['roaming'].value_counts()

roaming
False    1974
True     1938
Name: count, dtype: int64